# Mistral 7B Fine-Tuning with Professor Synapse Dataset
This notebook implements a QLoRA fine-tuning workflow for Mistral 7B using Unsloth AI optimized training and the Professor Synapse conversational dataset[1][3][5]. Key features include 4096 token context handling, reasoning block integration, and Colab T4 GPU optimization.

In [ ]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
!pip install datasets==2.18.0 trl==0.8.0 peft==0.10.0

In [ ]:
from unsloth import FastMistralModel

model, tokenizer = FastMistralModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
    token = "hf_...",
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("SynapticLabs/professor-synapse", split="train")

def format_conversation(example):
    messages = example["messages"]
    formatted = "<s>"
    
    for msg in messages:
        if msg["role"] == "user":
            formatted += "[INST] " + msg["content"] + " [/INST]"
        elif msg["role"] == "assistant":
            formatted += "<reasoning>" + msg.get("reasoning", "") + "</reasoning>"
            formatted += msg["content"] + "</s>"
    
    return {"text": formatted}

dataset = dataset.map(format_conversation, remove_columns=["messages"])

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    packing = True,
    args = {
        "num_train_epochs": 3,
        "per_device_train_batch_size": 2,
        "gradient_accumulation_steps": 4,
        "warmup_ratio": 0.1,
        "learning_rate": 2e-5,
        "fp16": not torch.cuda.is_bf16_supported(),
        "bf16": torch.cuda.is_bf16_supported(),
        "logging_steps": 50,
        "optim": "adamw_8bit",
        "weight_decay": 0.01,
        "lr_scheduler_type": "cosine",
        "seed": 42,
        "output_dir": "outputs",
    }
)

In [ ]:
trainer.train()
model.save_pretrained("professor_synapse_finetuned")